In [1]:
import os
import sys
import numpy as np
import pandas as pd
import scipy.stats as stats

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
import matplotlib.dates as mdates
from datetime import datetime, timedelta


print(sys.executable)

%matplotlib ipympl

color_array = [
    (0.121, 0.466, 0.705, 1.0),  # Muted blue
    (1.000, 0.498, 0.055, 1.0),  # Muted orange
    (0.172, 0.627, 0.172, 1.0),  # Muted green
    (0.839, 0.153, 0.157, 1.0),  # Muted red
    (0.580, 0.404, 0.741, 1.0),  # Muted purple
    (0.549, 0.337, 0.294, 1.0),  # Muted brown
    (0.890, 0.466, 0.760, 1.0),  # Muted pink
    (0.498, 0.498, 0.498, 1.0),  # Muted gray
    (0.737, 0.741, 0.133, 1.0),  # Muted yellow-green
]


/usr/bin/python3


In [2]:
class myWaterTest:
  def __init__(self, testingTime, cumTimeOffset=0, liquLabel = 'DI Water', liquConc = '0uL/100mL', testingSequence=[]):
    self.testingTime = testingTime
    self.cumTimeOffset = cumTimeOffset#-10
    self.liquLabel = liquLabel
    self.liquConc = liquConc
    self.csvFile = '/tf/data/waterSensor/aryData/sensor_'+testingTime+'.csv'
    self.sequenceCsvFile = '/tf/data/waterSensor/aryData/pump_sequence_'+testingTime+'.csv'
      
    #Sensor data
    colnames = ['time']+[f'px{i}' for i in range(9)]
    self.df=pd.read_csv(self.csvFile, names=colnames, header=None, skipfooter=1, engine='python')
    
    self.df['datetime'] = pd.to_datetime(self.df['time'], format='%Y-%m-%dT%H:%M:%S%z').dt.tz_localize(None)
    self.df = self.df.set_index(['datetime'])
    self.df = self.df.drop('time', axis=1)
    self.df.asfreq('s', method='ffill')

    #Control data
    self.seq_df=pd.read_csv(self.sequenceCsvFile, usecols=['Time(s)']+[f"Pump {i+1} (rpm)" for i in range(6)])
    seq_cycle = int(pd.read_csv(self.sequenceCsvFile)['Cycle'][0])
    self.seq_df = pd.concat([self.seq_df]*seq_cycle)
    self.seq_df['cumTime'] = self.seq_df['Time(s)'].cumsum(axis = 0)
    self.seq_df = self.seq_df.rename(columns=dict(zip([f"Pump {i+1} (rpm)" for i in range(6)], [f"Pump{i+1}" for i in range(6)])))
    
    self.seq_df = pd.DataFrame(
        data=self.seq_df.values.repeat(2, axis=0),  # Repeat each row twice
        columns=self.seq_df.columns                # Keep the column names
    )
    self.seq_df['cumTime'] = self.seq_df['cumTime'].shift(1).fillna(0)
    self.seq_df.loc[self.seq_df.index[1::2], 'cumTime'] = self.seq_df.loc[self.seq_df.index[1::2], 'cumTime'] - 1
    
    self.seq_df["timestamp"] = self.seq_df["cumTime"].apply(lambda x: self.df.index[0] + timedelta(seconds=x) if x >= 0 else None)
    self.seq_df.set_index("timestamp", inplace=True)
    
    #Set keyframe
    flowRatePt = 20
    self.keyTime = self.seq_df[(self.seq_df.Pump2>0)|(self.seq_df.Pump3>0)|(self.seq_df.Pump4>0)|(self.seq_df.Pump5>0)].index #(seq_df.Pump1==20)|
    self.keyTime = self.keyTime[1::2]
    self.keyTime = self.keyTime[1::2]
    # print(keyTime)

    flowRatePt = 100
    # self.keyTimeBaseline = self.seq_df[(self.seq_df.Pump2==flowRatePt)|(self.seq_df.Pump3==flowRatePt)|(self.seq_df.Pump4==flowRatePt)|(self.seq_df.Pump5==flowRatePt)].index
    # self.keyTimeBaseline = self.seq_df[(self.seq_df.Pump1==100)].index
    # self.keyTimeBaseline = self.keyTimeBaseline[::2]
    # print(keyTimeBaseline)

    self.testingSequence = testingSequence*seq_cycle #uL

In [ ]:
myTest = []

testingSequence = [10, 30, 60, 100] #uL:1L
myTest.append(myWaterTest('20250603_125215', liquLabel = 'Acetone', liquConc = '10uL/100mL', testingSequence=testingSequence))
myTest.append(myWaterTest('20250603_172217', liquLabel = 'Benzene', liquConc = '10uL/100mL', testingSequence=testingSequence))
myTest.append(myWaterTest('20250603_202421', liquLabel = 'Toluene', liquConc = '10uL/100mL', testingSequence=testingSequence))
myTest.append(myWaterTest('20250603_232616', liquLabel = 'Formaldehyde', liquConc = '10uL/100mL', testingSequence=testingSequence))

In [3]:
!ls


capstone.ipynb	plotTool	   water3x3_classifi.ipynb
exportCsv	tempplateKeyf.csv  water3x3_processing.ipynb
exportSvg	untitled.txt	   water3x3_si_clssifi.ipynb
